In [1]:
import pandas as pd
import DB_Utilities
from  File_Utilities import FileTools
FileTools.MYDIR = "cbp_data"
DBTools = DB_Utilities.DBTools()  # instantiate the class

redownload = False
send_to_db = False

In [2]:
# FileTools.check_file(FileTools.get_full_file_path(FileTools.get_filename_from_url("https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp20co.zip")))
# FileTools.check_file_from_url("https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp20co.zip")

In [3]:
year = "2020"

def get_file_by_year(year):
    if redownload:
        FileTools.unzip_file(FileTools.get_file(f"https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp{year[2:]}co.zip"))
        FileTools.unzip_file(FileTools.get_file(f"https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp{year[2:]}st.zip"))
        FileTools.unzip_file(FileTools.get_file(f"https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp{year[2:]}us.zip"))

    df_county = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}co.txt'))
    df_state = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}st.txt'))
    df_us = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}us.txt'))

    if(send_to_db):
        DBTools.insert_df(df_county, "cbp_county")
        DBTools.insert_df(df_state, "cbp_state")
        DBTools.insert_df(df_us, "cbp_us")


    return df_county, df_state, df_us



In [21]:
def munge_data(_df):
    _df['naics'] = _df['naics'].str.replace('-', '')
    _df['naics'] = _df['naics'].str.replace(' ', '')
    _df['naics'] = _df['naics'].str.replace('/', '')
    _df['naics_level'] = _df['naics'].str.len()
    try:
        # If there is no county_fips just don't do this. 
        # This means that this data is coming from either the state or the us.
        _df['county_fips'] = _df['fipstate'].astype(str).str.zfill(2)+_df['fipscty'].astype(str).str.zfill(3)    
    except:
        pass
    return _df

,fipstate,fipscty,naics,emp_nf,emp_county_naics,qp1_nf,qp1,ap_nf,ap,est,...,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,naics_level,county_fips,emp_county,percent_of_county_emp
0,1,1,,G,11265,G,94865,G,385785,879,...,N,N,N,N,N,N,0,01001,11265,1.000000
1,1,1,11,H,92,G,1183,H,5232,10,...,N,N,N,N,N,N,2,01001,11265,0.008167
2,1,1,113,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,3,01001,11265,0.007279
3,1,1,1133,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,4,01001,11265,0.007279
4,1,1,11331,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,5,01001,11265,0.007279


,fipstate,naics,percent_of_state_emp,emp_state_naics,emp_state
0,1,,1.000000,1777495,1777495
1,1,11,0.003589,6380,1777495
2,1,113,0.002256,4010,1777495
3,1,1131,0.000149,264,1777495
4,1,11311,0.000149,264,1777495


,naics,percent_of_us_emp,emp_us_naics,emp_us
0,,1.000000,134163349,134163349
1,11,0.001246,167204,134163349
2,113,0.000378,50710,134163349
3,1131,0.000027,3643,134163349
4,11311,0.000027,3643,134163349


In [ ]:
def run_all(year=2020):
   df_county, df_state, df_us =  get_file_by_year(year)

   df_county = munge_data(df_county)
   # We only want top level employmnet data at the state and us level.
   # LFO             C       Legal Form of Organization

   #                         '-' - All Establishments                        
   #                         C - C-Corporations and other corporate legal forms of organization
   #                         Z - S-Corporations
   #                         S - Sole Proprietorships
   #                         P - Partnerships
   #                         N - Non-Profits
   #                         G - Government
   #                         O - Other

   df_state = munge_data(df_state)
   df_state = df_state[df_state.lfo == '-'] #Only All Establishments
   df_us = munge_data(df_us)
   df_us = df_us[df_us.lfo == '-'] #Only All Establishments


   # county level employment, we will join on this later as our base data
   df_county_emp = df_county[df_county.naics_level == 0][['county_fips','emp']]
   # state level employment, we need this to calculate the locaton quotient in relation to the state.
   df_state_emp = df_state[(df_state.naics_level == 0) ][['fipstate','emp']]
   # us level employment, we need this to calculate the locaton quotient in relation to the us.
   df_us_emp = df_us[(df_us.naics_level == 0) ][['uscode','emp']]


   # Get the percent of each naics code in each county
   emp_percent_by_county = pd.merge(df_county, df_county_emp, on="county_fips")
   emp_percent_by_county['percent_of_county_emp'] = emp_percent_by_county['emp_x']/emp_percent_by_county['emp_y']
   emp_percent_by_county.rename(columns = {'emp_x':'emp_county_naics' , 'emp_y':'emp_county'}, inplace = True)

   # Get the percent of each naics code in each state
   emp_percent_by_state = pd.merge(df_state, df_state_emp, on="fipstate")
   emp_percent_by_state['percent_of_state_emp'] = emp_percent_by_state['emp_x']/emp_percent_by_state['emp_y']
   emp_percent_by_state.rename(columns = {'emp_x':'emp_state_naics', 'emp_y':'emp_state' }, inplace = True)
   emp_percent_by_state = emp_percent_by_state[['fipstate','naics','percent_of_state_emp', 'emp_state_naics', 'emp_state']]

   # Get the percent of each naics code in the US
   emp_percent_by_us = pd.merge(df_us, df_us_emp, on="uscode")
   emp_percent_by_us['percent_of_us_emp'] = emp_percent_by_us['emp_x']/emp_percent_by_us['emp_y']
   emp_percent_by_us.rename(columns = {'emp_x':'emp_us_naics', 'emp_y':'emp_us' }, inplace = True)
   emp_percent_by_us = emp_percent_by_us[['naics','percent_of_us_emp', 'emp_us_naics', 'emp_us']]
   emp_percent_by_us.head()

      # Merge all the data together so we can calculate the location quotient.
   emp_percent_by_county_state = pd.merge(emp_percent_by_county, emp_percent_by_state, how='left', left_on=["naics", 'fipstate'], right_on=["naics", 'fipstate'])
   emp_percent_by_county_state_us = pd.merge(emp_percent_by_county_state, emp_percent_by_us, how='left', left_on=["naics"], right_on=["naics"])

   # Calculate the location quotient for county/state and for county/us
   emp_percent_by_county_state_us['location_quotient_county_state'] = emp_percent_by_county_state_us['percent_of_county_emp']/emp_percent_by_county_state_us['percent_of_state_emp']
   emp_percent_by_county_state_us['location_quotient_county_us'] = emp_percent_by_county_state_us['percent_of_county_emp']/emp_percent_by_county_state_us['percent_of_us_emp']

   if(send_to_db):
      DBTools.insert_df(emp_percent_by_county_state_us, f"cbp_emp_percent_by_county_state_us_{year}")